# Navigation

---

*Jun Zhu, zhujun981661@gmail.com, 07.2020*

In this notebook, I present the solution for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).


### 1. Start the Environment

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
brain_name, brain

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Run the environment with random actions

In [ ]:
def play(env, agent=None):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    score = 0
    
    if agent is None:
        while True:
            action = np.random.randint(action_size)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            score += reward
            state = next_state
            if done:
                break
    else:
        agent.play(env)

    print("Score: {}".format(score))

### 4. Train an agent with deep-Q network (DQN)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from dqn_agent import DqnAgent

In [ ]:
class BananaBrainQNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        state_space = 37
        action_space = 4
        self._fc1 = nn.Linear(state_space, 64)
        self._fc2 = nn.Linear(64, 128)
        self._fc_final = nn.Linear(128, action_space)

    def forward(self, state):
        x = state
        for fc in (self._fc1, self._fc2):
            x = F.relu(fc(x))
        return self._fc_final(x)
    

In [ ]:
actions = np.arange(4)
model = BananaBrainQNetwork()
dqn_agent = DqnAgent(model, actions, batch_size=64, replay_memory_size=50000, 
                     learning_rate=5e-4, discount_factor=0.99)

dqn_agent.train(env, max_steps=500, n_episodes=2000, output_frequency=10, target_score=13)

In [ ]:
dqn_agent.play(env)

In [ ]:
# env.close()